In [1]:
import pathlib as pl
import pandas as pd
import numpy as np
import time
import imperial_materials_simulation as ims

### Physics Module Tests

In [2]:
n_atoms = 22
equilibrium_bond_length = 1.53
spring_constant = 15.18
sigma = 4.5
epsilon = 0.00485678
positions = np.zeros(shape=(n_atoms, 3))
positions[:, 1] = np.linspace(0, (equilibrium_bond_length+np.random.rand()/10)-1, num=n_atoms)

In [5]:
def GetBondingEnergy(pos, calculate_force=True): #old function
#
    potential_bond = 0.0
#
    bond = np.zeros(3)
    bonddir = np.zeros(3)
    force_bond = np.zeros((n_atoms,3))
#
    for i in range(n_atoms-1):
        bond = pos[i+1]-pos[i]
        bondlength = np.linalg.norm(bond)
        bonddir = bond/bondlength
        dbondlength = bondlength - equilibrium_bond_length
        potential_bond = potential_bond + spring_constant*(dbondlength**2.0)/2.0
        if calculate_force:
            force_bond[i] = force_bond[i] + spring_constant*dbondlength*bonddir
            force_bond[i+1] = force_bond[i+1] - spring_constant*dbondlength*bonddir
    
    return potential_bond, force_bond

In [6]:
my_forces, my_potential = phys.get_bonding_interactions(positions, equilibrium_bond_length, spring_constant)
his_potential, his_forces = GetBondingEnergy(positions)
assert np.allclose(my_potential, his_potential)
assert np.allclose(my_forces, his_forces)

In [7]:
# start = time.perf_counter()
# for i in range(100_000):
#     GetBondingEnergy(positions)
# old_run_time = time.perf_counter() - start
# old_run_time

In [8]:
# start = time.perf_counter()
# for i in range(100_000):
#     phys.get_bonding_interactions(positions, equilibrium_bond_length, spring_constant)
# new_run_time = time.perf_counter() - start
# new_run_time

In [9]:
# old_run_time / new_run_time

In [10]:
def GetNonBondingEnergy(pos, calculate_force=True): #old function
#
    disp = np.zeros(3)
    dforce = np.zeros(3)
    force_nonbond = np.zeros((n_atoms,3))
#
    potential_nonbond = 0.0
    factor1 = -12.0*epsilon/(sigma*sigma)
    for i in range(n_atoms-1):
        for j in range(i+3,n_atoms):
            disp = pos[j]-pos[i]
            dist = np.linalg.norm(disp)
            squared = (sigma/dist)**2.0
            sixpower = squared**3.0 
            twelvepower = sixpower*sixpower
            potential_nonbond = potential_nonbond + epsilon*(twelvepower - 2.0*sixpower)
            if calculate_force:
                dforce  = factor1*squared*(twelvepower - sixpower)*disp
                force_nonbond[i] = force_nonbond[i] + dforce
                force_nonbond[j] = force_nonbond[j] - dforce
    return potential_nonbond, force_nonbond

In [11]:
def GetNonBondingEnergy_Carlos(pos, calculate_force=None):
   #
      force_nonbond = np.zeros((n_atoms,3))
   #
      potential_nonbond = 0.0
      factor1 = -12.0*epsilon/(sigma*sigma)
      for i in range(n_atoms-1):
         
         disp_mat=pos[i+3:]-pos[i]
         #dist_sq=calc_dist(pos[i+3:],pos[i].reshape((1,3))).flatten()
         #potentials
         dist_sq=np.sum(disp_mat*disp_mat, axis=1)
         squared_mat = np.divide(sigma**2,dist_sq)
         sixpower_mat = squared_mat**3.0 
         twelvepower_mat = sixpower_mat*sixpower_mat
         potentials_nonbond_mat = epsilon*(twelvepower_mat - 2.0*sixpower_mat)
         potential_nonbond = potential_nonbond+potentials_nonbond_mat.sum()
         
         
         #
         dforce=disp_mat*(factor1*squared_mat*(twelvepower_mat - sixpower_mat))[:,None]
         force_nonbond[i]=force_nonbond[i]+np.sum(dforce,axis=0)
         force_nonbond[i+3:]=force_nonbond[i+3:]- dforce

      return potential_nonbond, force_nonbond

In [12]:
my_forces, my_potential = phys.get_non_bonding_interactions(positions, epsilon, sigma)
his_potential, his_forces = GetNonBondingEnergy(positions)
assert np.allclose(my_potential, his_potential)
assert np.allclose(my_forces, his_forces)

In [13]:
# start = time.perf_counter()
# for i in range(50_000):
#     GetNonBondingEnergy_Carlos(positions)
# old_run_time_nb = time.perf_counter() - start
# old_run_time_nb

In [14]:
# start = time.perf_counter()
# for i in range(50_000):
#     phys.get_non_bonding_interactions(positions, epsilon, sigma)
# new_run_time_nb = time.perf_counter() - start
# new_run_time_nb

In [15]:
# old_run_time_nb / new_run_time_nb

In [16]:
class OldPotentialEnergyTracker():
    '''
    Utility class for efficiently tracking how total potential energy (bonding + Lennard Jones non-bonding) changes
    when a single atom is displaced.
    '''

    def __init__(self, positions: np.ndarray, epsilon: float, sigma: float, equilibrium_bond_length: float,
                spring_constant: float) -> None:
        'calculates key distances, lengths and total potential energy'
        self.positions = positions
        self.epsilon = epsilon
        self.sigma = sigma
        self.equilibrium_bond_length = equilibrium_bond_length
        self.spring_constant = spring_constant
        self.n_atoms = len(positions)

        # (n_atoms, n_atoms, 3) array where array[i,j] is the displacement between ith atom and jth atom
        self.displacements = positions.reshape((self.n_atoms, 1, 3)) - positions.reshape((1, self.n_atoms, 3))
        
        self.col_indexes, self.row_indexes = np.meshgrid(np.arange(self.n_atoms), np.arange(self.n_atoms))
        self.bonding_mask = (self.col_indexes-self.row_indexes) == 1 #all bonding interactions
        #half of non-bonding interactions (all that is needed to calculate non bonding potential energy)
        self.non_bonding_mask = (self.col_indexes-self.row_indexes) > 2 

        #should ignore distances between atom and itself (i==j). Set to 1 to prevent 0 division errors
        self.displacements[self.col_indexes==self.row_indexes] = 1
        self.lengths = np.linalg.norm(self.displacements, axis=2)

        bonding_extensions = self.lengths[self.bonding_mask] - equilibrium_bond_length
        bonding_potential =  np.sum(spring_constant/2 * bonding_extensions**2)
        six_power = (sigma/self.lengths[self.non_bonding_mask]) ** 6
        non_bonding_potential = np.sum(epsilon * (six_power**2 - 2*six_power))
        self.total_potential_energy = bonding_potential + non_bonding_potential
        
        #used in test_displacement method. allows for each pair to only be checked once. length between ij
        #calculated, but not length between ji. using the second length when only calculating potentials is redundant
        self.relevant_interactions = self.col_indexes > self.row_indexes

    def get_total_potential_energy(self):
        '''return total potential energy of molecule'''
        return self.total_potential_energy

    def test_displacement(self, atom_index: int, displacement: np.ndarray) -> float:
        '''stores temporary new positions, displacements, and lengths. Returns change in potential energy'''
        self.new_positions = self.positions.copy()
        self.new_positions[atom_index] += displacement

        self.new_displacements = self.displacements.copy()
        self.new_displacements[:, atom_index] = self.new_positions - self.new_positions[atom_index].reshape(1, 3)
        self.new_displacements[atom_index, :] = self.new_positions[atom_index].reshape(1, 3) - self.new_positions

        affected_interactions = (self.col_indexes == atom_index) | (self.row_indexes == atom_index)
        update_mask = self.relevant_interactions & affected_interactions
        self.new_lengths = self.lengths.copy()
        #axis=1 as slicing 3D array with boolean mask returns 2D array
        self.new_lengths[update_mask] = np.linalg.norm(self.new_displacements[update_mask], axis=1)

        bonding_extensions = self.new_lengths[self.bonding_mask] - self.equilibrium_bond_length
        bonding_potential =  np.sum(self.spring_constant/2 * bonding_extensions**2)
        six_power = (self.sigma/self.new_lengths[self.non_bonding_mask])**6
        non_bonding_potential = np.sum(self.epsilon * (six_power**2 - 2*six_power))
        self.new_total_potential_energy = bonding_potential + non_bonding_potential
        return self.new_total_potential_energy - self.total_potential_energy
    
    def accept_last_displacement(self) -> None:
        '''replaces internal positions, displacements, and lengths with values from last test displacement'''
        self.positions = self.new_positions
        self.displacements = self.new_displacements
        self.lengths = self.new_lengths
        self.total_potential_energy = self.new_total_potential_energy

In [17]:
F_b, PE_b = phys.get_bonding_interactions(positions, equilibrium_bond_length, spring_constant)
F_nb, PE_nb = phys.get_non_bonding_interactions(positions, epsilon, sigma)
func_PE = PE_b + PE_nb

energy_tracker = phys.PotentialEnergyTracker(positions, epsilon, sigma, equilibrium_bond_length, spring_constant)
class_PE = energy_tracker.get_total_potential_energy()

assert np.allclose(func_PE, class_PE)

In [18]:
atom_index, displacement = 1, np.array([0.1, 1, -0.3    ])
new_positions = positions.copy()
new_positions[atom_index] += displacement
new_F_b, new_PE_b = phys.get_bonding_interactions(new_positions, equilibrium_bond_length, spring_constant)
new_F_nb, new_PE_nb = phys.get_non_bonding_interactions(new_positions, epsilon, sigma)
new_func_PE = new_PE_b + new_PE_nb
func_PE_change = new_func_PE - func_PE

class_PE_change = energy_tracker.test_displacement(atom_index, displacement)

assert np.allclose(func_PE_change, class_PE_change)

In [20]:
# energy_tracker = OldPotentialEnergyTracker(positions, epsilon, sigma, equilibrium_bond_length, spring_constant)
# start = time.perf_counter()
# for i in range(50_000):
#     energy_tracker.test_displacement(atom_index, displacement)
# old_MMC_runtime = time.perf_counter() - start
# old_MMC_runtime

2.733192899962887

In [21]:
# energy_tracker = phys.PotentialEnergyTracker(positions, epsilon, sigma, equilibrium_bond_length, spring_constant)
# start = time.perf_counter()
# for i in range(50_000):
#     energy_tracker.test_displacement(atom_index, displacement)
# new_MMC_runtime = time.perf_counter() - start
# new_MMC_runtime

0.30041690001962706

In [22]:
# old_MMC_runtime / new_MMC_runtime

9.097999812208702

### Main Module Tests

In [ ]:
sim = main.Simulation(n_atoms=22, starting_temperature=1000)
sim.NVT_run(n_steps=50_000, temperature=1200)
sim.relax_run(n_steps=50_000)

#206.5s paul non bonding
#88.9s carlos non bonding
#20.7s ayham numpy without linalg norm
#20.6s ayham numpy
#19.7 ayham numpy with njit paul bonding
#18.5 ayham numpy with bonding njit
#13.9s paul bonding parallel njit and njit carlos non bonding
#12.1s njit (paul non bonding)
#11.2s njit (carlos non bonding)
#9.1s ayham numpy bonding njit and njit carlos non bonding
#8.0s numpy bonding njit fastmath and njit fastmath carlos non bonding

In [3]:
sim = main.Simulation(n_atoms=22, starting_temperature=1000)
sim.MMC_run(n_steps=500_000, temperature=2000)

#11.0s for compiled numpy calculate PE change solution with precalculated random displacements
#15.6s for compiled numpy calculate PE change solution
#39.3s for old numpy recalculate full PE solution

500,000 step MMC run completed


### Data Logging Tests

In [ ]:
numbers = np.random.uniform(low=0, high=4, size=(20_000, 10))
columns = [f'col_{i}' for i in range(10)]

In [ ]:
start = time.perf_counter()
dataframe = pd.DataFrame(numbers, columns=columns)
stop = time.perf_counter()
print(stop-start)

0.0004496999899856746


In [ ]:
start = time.perf_counter()
dataframe = pd.DataFrame(np.full(shape=numbers.shape, fill_value=np.NAN), columns=columns)
for j, row in enumerate(numbers):
    for i, value in enumerate(row):
        dataframe.loc[j, f'col_{i}'] = value
stop = time.perf_counter()
print(stop-start)

10.02576119999867


In [ ]:
start = time.perf_counter()
dataframe = pd.DataFrame(np.full(shape=numbers.shape, fill_value=np.NAN), columns=columns)
for j, row in enumerate(numbers):
    for i, value in enumerate(row):
        dataframe.iloc[j, i] = value
stop = time.perf_counter()
print(stop-start)

7.962099500000477


In [ ]:
start = time.perf_counter()
array = np.full(shape=numbers.shape, fill_value=np.NAN)
for j, row in enumerate(numbers):
    for i, value in enumerate(row):
        array[j, i] = value
dataframe = pd.DataFrame(array, columns=columns)
stop = time.perf_counter()
print(stop-start)

0.11671750003006309


In [ ]:
start = time.perf_counter()
counter = 0
dataframe = pd.DataFrame(np.full(shape=numbers.shape, fill_value=np.NAN), columns=columns)
for j, row in enumerate(numbers):
    for i, value in enumerate(row):
        counter += 1
stop = time.perf_counter()
print(stop-start)

0.06050989998038858


In [ ]:
start = time.perf_counter()
dataframe = pd.DataFrame(np.full(shape=numbers.shape, fill_value=np.NAN), columns=columns)
for j, row in enumerate(numbers):
    dataframe.loc[j] = row
stop = time.perf_counter()
print(stop-start)

0.6060011000372469


In [ ]:
start = time.perf_counter()
dataframe = pd.DataFrame(np.full(shape=numbers.shape, fill_value=np.NAN), columns=columns)
for j, row in enumerate(numbers):
    row = {f'col_{i}': value for i, value in enumerate(row)}
    dataframe.loc[j] = row
stop = time.perf_counter()
print(stop-start)

3.166217799996957


In [ ]:
start = time.perf_counter()
rows = []
for j, row in enumerate(numbers):
    rows.append({f'col_{i}': value for i, value in enumerate(row)})
dataframe = pd.DataFrame(rows)
stop = time.perf_counter()
print(stop-start)

0.15096970001468435


In [ ]:
start = time.perf_counter()
rows = []
for j, row in enumerate(numbers):
    rows.append({f'col_{i}': value.copy() for i, value in enumerate(row)})
dataframe = pd.DataFrame(rows)
stop = time.perf_counter()
print(stop-start)

0.2644046999630518


In [ ]:
start = time.perf_counter()
with open('temp.txt', 'w') as file:
    for row in numbers:
        file.write(' '.join(map(str, row)) + '\n')
stop = time.perf_counter()
pl.Path().absolute().joinpath('temp.txt').unlink()
print(stop-start)

0.17122710001422092
